## LBox 데이터 크롤링

### 1. LBox 계정정보 가져오기
LBox에 로그인할 계정 정보를 가져옵니다. 계정 정보는 현재 디렉토리에 `config_lbox.ini` 파일을 생성한뒤 아래와 같이 작성합니다.

```ini
[ACCOUNT1]
email = 이메일1
password = 비밀번호1

[ACCOUNT2]
email = 이메일2
password = 비밀번호2
```

In [1]:
import configparser as parser

properties = parser.ConfigParser()
properties.read("./config_lbox.ini")

login_accounts = [properties['ACCOUNT1'], properties['ACCOUNT2']]

### 2. 데이터 크롤링

**기본 상수 설정**
* `LOGIN_TOKEN` : 사용자 로그인 세션을 증명하는 토큰입니다.

**법원 리스트**
* 서울중앙지방법원
* 서울동부지방법원
* 서울남부지방법원
* 서울북부지방법원
* 서울서부지방법원
* 의정부지방법원
* 인천지방법원
* 춘천지방법원
* 대전지방법원
* 청주지방법원
* 대구지방법원
* 부산지방법원
* 울산지방법원
* 창원지방법원
* 광주지방법원
* 전주지방법원
* 제주지방법원
* 수원지방법원

In [2]:
import requests
import uuid

############
# 로그인 토큰
############
# 데이터를 가져오기 위해 로그인 토큰을 얻습니다.
session = requests.session() # 로그인 데이터를 요청할 세션
device_id = str(uuid.uuid4()) # 디바이스 ID
account_index = 0 # 사용할 계정 인덱스

def get_login_token():
    login_response = session.post("https://lbox.kr/api/user/login?deviceId=not%20allowed%20"+device_id, json={"email":login_accounts[account_index]['email'], "password":login_accounts[account_index]['password']})
    return login_response.cookies.get_dict()['lboxToken']
login_token = get_login_token()

print("Your device id is", device_id)

Your device id is 8945c8b3-8e46-432a-a6f7-7b56c0f51dbc


In [5]:
from tqdm import trange # 프로그레스바 출력을 위한 라이브러리
import pandas as pd
import time
import random
import os
import xlsxwriter


#################
# 상수
#################
# 크롤링 관련 상수
# COURT_NAMES = ["대전지방법원", "청주지방법원", "대구지방법원", "부산지방법원", "울산지방법원", "창원지방법원", "광주지방법원", "전주지방법원", "제주지방법원", "수원지방법원"] # 크롤링할 법원명 리스트. 리스트 내에 있는 법원들만 크롤링함
COURT_NAMES = ["창원지방법원", "광주지방법원", "전주지방법원", "제주지방법원", "수원지방법원"] # 크롤링할 법원명 리스트. 리스트 내에 있는 법원들만 크롤링함
OUTPUT_PATH = "output/"     # 결과가 저장될 폴더 경로


###############
# 사용자 함수
###############
def save_to_excel(_dataframe, court_name):
    _dataframe.to_excel(os.path.join(OUTPUT_PATH, "data_{}.xlsx".format(court_name)), index=False, engine = "xlsxwriter")
    print("Output file : " + os.path.join(OUTPUT_PATH, "data_{}.xlsx".format(court_name)))


#############
# 초기화
#############
# 결과 디렉토리가 없으면 생성
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)


##############
# 크롤링
##############
for (court_index, court_name) in enumerate(COURT_NAMES):
    request_payload = {"query":"도로교통법위반","page":0,"size":10,"caseType":"criminal","courtType":"","trialType":"판결","data":court_name,"range":"3"} # 기본 요청 데이터

    # 총 판례 수 가져오기
    case_list_response = session.post("https://lbox.kr/api/case/search", json=request_payload).json()
    number_of_precedents = int(case_list_response['count'])
    
    # 총 페이지 수 계산
    number_of_pages = (number_of_precedents // 10) + (0 if number_of_precedents % 10 == 0 else 1)

    print("[{} / {}] {} {}개".format(court_index + 1, len(COURT_NAMES), court_name, number_of_precedents))

    # 판례 데이터 크롤링
    dataframe = pd.DataFrame() # 최종 데이터가 저장될 데이터프레임
    for page in trange(number_of_pages):
        # 페이지 변경
        request_payload["page"] = page
        case_list_response = session.post("https://lbox.kr/api/case/search", json=request_payload).json()

        # 해당 페이지의 판례들 가져오기
        for case in case_list_response["caseList"]:
            time.sleep(0.5) # 잠시 대기

            # 캡챠 없이 정상적으로 크롤링할 수 있을 때만 진행
            crawling_status = False
            while not crawling_status:
                try:
                    case_response = session.get("https://lbox.kr/api/case?id={}&searchKeyword=도로교통법위반".format(case['id']), cookies={"lboxToken": login_token}).json()
                    crawling_status = True
                except:
                    # 계정 전환
                    try:
                        # 다른 계정으로 새로운 로그인 토큰을 얻음
                        account_index = 1 if account_index == 0 else 0
                        login_token = get_login_token()
                        # 다시 크롤링 진행
                        case_response = session.get("https://lbox.kr/api/case?id={}&searchKeyword=도로교통법위반".format(case['id']), cookies={"lboxToken": login_token}).json()
                        crawling_status = True
                    except:
                        user_input = input("Try again? ('c' for exit)")
                        if user_input == 'c':
                            save_to_excel(dataframe, court_name)
                            raise

            # 사건번호
            case_number = case_response['case']['doc_id'][case_response['case']['doc_id'].index("-") + 1:]

            # 사건결과
            case_result = case['result']

            # 위반 그룹 (여러개일 수 있음)
            violation_groups = case_response['case']['casename_groups']

            # 위반 내용
            violations = case_response['case']['casename_list']

            # 판결선고, 주문, 이유, 변호인
            judgement_date, judgement_decision, judgement_reason, attorney_name = "", "", "", ""
            for section in case_response['case']['section_list']:
                if section['title'] == "판결선고":
                    judgement_date = section['text']
                elif section['title'] == "주문":
                    judgement_decision = section['text']
                elif section['title'] == "이유":
                    judgement_reason = section['text']
                elif section['title'] == "변호인":
                    attorney_name = section['text']

            # 판사 이름
            judge_name = case_response['case']['judges']

            # 증거 URL
            evidence_urls = case_response['case']['evidence_list']


            # 임시 데이터프레임 생성 및 저장
            _dataframe = pd.DataFrame({
                "court_name" : court_name,
                "case_number" : case_number,
                "case_result" : case_result,
                "violation_groups" : [violation_groups],
                "violations" : [violations],
                "judgement_date" : judgement_date,
                "judgement_decision" : judgement_decision,
                "judgement_reason" : judgement_reason,
                "judge_name" : judge_name,
                "attorney_name" : attorney_name,
                "evidence_urls" : [evidence_urls]
            })
            dataframe = pd.concat([dataframe, _dataframe])
    # 데이터 파일로 저장
    save_to_excel(dataframe, court_name)

  0%|          | 0/879 [00:00<?, ?it/s]

[1 / 5] 창원지방법원 8790개


100%|██████████| 879/879 [2:20:30<00:00,  9.59s/it]  


Output file : output/data_창원지방법원.xlsx


  0%|          | 0/950 [00:00<?, ?it/s]

[2 / 5] 광주지방법원 9495개


  0%|          | 0/950 [00:01<?, ?it/s]


KeyboardInterrupt: 